In [ ]:
import numpy as np
import xarray as xr

This is an example notebook for AVIRIS radiance data. The dataset is available at: https://aviris.jpl.nasa.gov/dataportal/.

In [ ]:
# The datasets are in the form of hierarchically-nested netCDF files, which cannot be opened as a standard xarray dataset.
# The files can only be opened as a datatree structure, which is only available in newer versions of xarray.
path = 'datasets\AV320230710t193100_016_L1B_RDN_cbeae6f8_RDN.nc'
ds = xr.open_datatree(path)
ds

In [ ]:
## Retrieve radiance information & assign arbitrary coordinates
samples_coords = np.arange(1234)
lines_coords = np.arange(1280)
radiance = ds.radiance.radiance.assign_coords({'samples':samples_coords, 'lines':lines_coords})
radiance

In [ ]:
radiance.sel(wavelength='2400', method='nearest').plot()

In [ ]:
# Plot the spectrum of a single pixel
radiance.sel(lines=600, samples=300).plot()